# Computer Infrastructure

This document will contain solutions/workings for the Computer Infrastructure module, the tasks completed in this are from the [problems.md](https://github.com/ianmcloughlin/computer-infrastructure/blob/main/assessment/problems.md) document for this module.
The assessment details can be found in [assessment.md](https://github.com/ianmcloughlin/computer-infrastructure/blob/main/assessment/assessment.md) and in the [guidance notebook](https://github.com/ianmcloughlin/computer-infrastructure/blob/main/assessment/guidance.ipynb).

Splitting this notebook into a number of different sections so that the problems can be completed consecutively.

## Problem 1: Getting the data

FROM THE TASK OUTLINE

What this part of the project involves:

Using the yfinance Python package, write a function called get_data() that downloads all hourly data for the previous five days for the five FAANG stocks:

- Facebook (META)
- Apple (AAPL)
- Amazon (AMZN)
- Netflix (NFLX)
- Google (GOOG)

See [https://www.youtube.com/watch?v=SxIwqdedomg](https://www.youtube.com/watch?v=SxIwqdedomg) for yfinance installation.

The function should save the data into a folder called data in the root of your repository using a filename with the format YYYYMMDD-HHmmss.csv where YYYYMMDD is the four-digit year (e.g. 2025), followed by the two-digit month (e.g. 09 for September), followed by the two digit day, and HHmmss is hour, minutes, seconds. Create the data folder if you don't already have one.

For the naming of the folder: [https://forum.enterprisedna.co/t/adding-a-datetime-to-the-name-of-the-file/45592/4](https://forum.enterprisedna.co/t/adding-a-datetime-to-the-name-of-the-file/45592/4) and [https://community.jmp.com/t5/Discussions/Format-Date-Time-yyyymmdd-hhmmss/td-p/694172](https://community.jmp.com/t5/Discussions/Format-Date-Time-yyyymmdd-hhmmss/td-p/694172)

For this project the [yfinance](https://github.com/ranaroussi/yfinance/blob/main/README.md) package will be used. 

From this, all hourly data for the previous 5 days for the 5 FAANG stocks (Facebook, Apple, Amazon, Netflix, Google) will be downloaded and saved into a folder, called data, in the root repository. File names will use the following format: YYYYMMDD-HHmmss.csv where YYYYMMDD is the four-digit year (e.g. 2025), followed by the two-digit month (e.g. 09 for September), followed by the two digit day, and HHmmss is hour, minutes, seconds.

In [ ]:
# First need to make sure the correct modules are installed.
# I decided to add all of these libraries because they are commonly used for data analysis and visualization and are allowed according to the requirements.
# See: https://github.com/ianmcloughlin/computer-infrastructure/blob/main/requirements.txt

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import yfinance as yf
import datetime
import statsmodels as sm
import scipy as spy

Importing the data and making sure to name it as appropriate in order to save it in an easily understandable and readable manner.

~~ DELETE~~
art 1 to do :

Using the yfinance Python package, write a function called get_data() that downloads all hourly data for the previous five days for the five FAANG stocks:

- Facebook (META)
- Apple (AAPL)
- Amazon (AMZN)
- Netflix (NFLX)
- Google (GOOG)

Need to first find out how to download the data for the FAANG stocks. All the hourly data for the past 5 days is required so this will be part of the process. See the [stack overflow page](https://stackoverflow.com/questions/74479906/how-to-get-aggregate-4hour-bars-historical-stock-data-using-yfinance-in-python) for an example of collecting data over the last 2 years hourly - this will be adapted to be for the last 5 days and hourly. When importing the data and make sure that it's labelled appropriately so that each of the 5 FAANG can be differentiated from one another. 

In [ ]:
# This code downloads the hourly data for the last 5 days for the 5 FAANG stocks (Facebook, Apple, Amazon, Netflix, Google) using the yfinance package.

faang5_data = yf.download(tickers="AAPL AMZN META GOOG NFLX", period="5d", interval="1h") # adapted from https://stackoverflow.com/questions/74479906/how-to-get-aggregate-4hour-bars-historical-stock-data-using-yfinance-in-python

def get_data(faang5_data): 
    print(faang5_data) # to check it works
    return faang5_data

# Now need to call the function to get the data
get_data(faang5_data)


make sure that the files are saving to the correct location. I accidentally saved the data into a new folder that was in the higher file on my pc so I decided that an absolute path was the best way to go to minimise problems (I'll to the same with the plots later).

In [ ]:
# I kept getting an error that the folder didn't exist so I created it manually in the root repository.
# So now I need to check that the directory exists before trying to save the file.
import os
output_dir = r"D:\Data_Analytics\Modules\CI\computer-infrastructure\data-faang-stocks"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
    # Get the absolute path of the output directory
absolute_output_dir = os.path.abspath(output_dir)

Save data as csv in correct format name

In [ ]:
# Now to save it as a .csv file in a folder called data-faang-stocks in the root repository with the correct naming format YYYYMMDD-HHmmss.csv.
# Save the data to a CSV file
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
file_path = os.path.join(absolute_output_dir, f"{current_time}.csv")
faang5_data.to_csv(file_path, index=False)

Verify save file

In [ ]:
# Verify the file was saved
print(f"File saved at: {file_path}")
print(f"Absolute path to output directory: {absolute_output_dir}")
print("Files in directory:", os.listdir(output_dir))

## Problem 2: Plotting Data

FROM THE TASK OUTLINE

What this problem involves:

Write a function called plot_data() that opens the latest data file in the data folder and, on one plot, plots the Close prices for each of the five stocks. The plot should include axis labels, a legend, and the date as a title. The function should save the plot into a plots folder in the root of your repository using a filename in the format YYYYMMDD-HHmmss.png. Create the plots folder if you don't already have one.

**I can use stuff from the PANDS project from term 1 for this to show saving images and exporting stuff to folders [HERE](https://github.com/KaiiMenai/pands-project), specifically in the [analysis.py](https://github.com/KaiiMenai/pands-project/blob/main/analysis.py) program.**

## Problem 3: Script

FROM THE TASK OUTLINE

What this problem involves:

Create a Python script called faang.py in the root of your repository. Copy the above functions into it and it so that whenever someone at the terminal types ./faang.py, the script runs, downloading the data and creating the plot. Note that this will require a shebang line and the script to be marked executable. Explain the steps you took in your notebook.

**AGAIN, refer to the PANDS project for the function for aid in the run and save parts of the script.**

## Problem 4: Automation

FROM THE TASK OUTLINE

What this problem involves:

Create a GitHub Actions workflow to run your script every Saturday morning. The script should be called faang.yml in a .github/workflows/ folder in the root of your repository. In your notebook, explain each of the individual lines in your workflow.

References to help with this task:

- [GitHub Actions workflow](https://docs.github.com/en/actions)
- [CI/CD Tutorial using GitHub Actions - Automated Testing & Automated Deployments](https://youtu.be/YLtlz88zrLg) on YT
- [trigger GitHub Actions on every Saturday](https://stackoverflow.com/questions/75974691/how-can-i-trigger-github-actions-on-every-3rd-saturday-of-a-month) on StackOverflow
- [Run on a schedule](https://jasonet.co/posts/scheduled-actions)


# END